### Code Flow and Explanation:

1. **Import Libraries**:
   - `pandas` for data manipulation.
   - `cosine_similarity` from `sklearn.metrics.pairwise` for similarity computation.
   - `numpy` for numerical operations.

2. **Load Data**:
   - Load the interaction data (`Combined_Interactions.xlsx`), event details (`Events.xlsx`), and user responses (`(Responses) (1).csv`).

3. **Item-User Interaction Matrix**:
   - Pivot the interaction data into an **item-user matrix** where rows are event IDs, columns are user names, and values are interaction scores. Missing values are filled with 0.

4. **Compute Item Similarity**:
   - Use cosine similarity to calculate the **item-item similarity matrix** based on users’ interactions with events.

5. **Item-Based Recommendation Function**:
   - The `item_based_recommend_events(user, top_n=10)` function recommends events for a user based on the events they have already interacted with. The steps include:
     - For each event a user has interacted with, find similar events.
     - Ensure that the recommended events have not already been interacted with by the user.
     - Accumulate recommendation scores based on item similarity.
     - Return the top `n` recommendations.

6. **Hybrid Recommendation Function**:
   - The `recommend_events(user, events_df, responses, top_n=10)` function provides recommendations using both **collaborative filtering** (item-based) and **content-based filtering**:
     - **Collaborative Filtering**:
       - Uses item-based recommendations from the `item_based_recommend_events()` function.
     - **Content-Based Filtering**:
       - Matches the user's interests (from responses) to the event categories (primary and secondary).
       - Events are filtered based on whether their categories match the user's interests.
       - Recommendations are expanded to cover subcategories not interacted with by the user.
     - **Final Recommendations**:
       - Add unrecommended events from categories the user has not yet interacted with.
       - Incorporate additional events based on interaction scores for further variety.
       - Return the combined list of recommended events.

7. **Example Usage**:
   - The function is called for the user `Pratik Bhande` to generate the top 10 event recommendations, which are then displayed.

In [4]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load data
interactions_df = pd.read_excel('Combined_Interactions.xlsx')
events_df = pd.read_excel('Events.xlsx')
responses = pd.read_csv('(Responses) (1).csv')

# Create item-user interaction matrix
item_user_matrix = interactions_df.pivot_table(index='Event ID', columns='Name', values='Interaction Score', fill_value=0)

# Compute item-item similarity matrix using cosine similarity
item_similarity = cosine_similarity(item_user_matrix)

# Function to recommend events for a given user based on item similarity
def item_based_recommend_events(user, top_n=10):
    # Get the events already interacted with by the user
    interacted_events = interactions_df[interactions_df['Name'] == user]['Event ID'].tolist()
    
    # Initialize dictionary to store event recommendations
    event_recommendations = {}

    # Iterate over the interacted events and recommend similar events
    for event_id in interacted_events:
        if event_id not in item_user_matrix.index:
            continue
        event_index = item_user_matrix.index.get_loc(event_id)
        similar_events = np.argsort(item_similarity[event_index])[::-1]  # Sort events by similarity
        
        for similar_event in similar_events:
            similar_event_id = item_user_matrix.index[similar_event]
            if similar_event_id not in interacted_events:
                if similar_event_id not in event_recommendations:
                    event_recommendations[similar_event_id] = item_similarity[event_index][similar_event]
                else:
                    event_recommendations[similar_event_id] += item_similarity[event_index][similar_event]
    
    # Sort recommendations by similarity score
    sorted_recommendations = sorted(event_recommendations.items(), key=lambda x: x[1], reverse=True)
    
    # Return top n recommendations
    return sorted_recommendations[:top_n]

# Hybrid recommendation function
def recommend_events(user, events_df, responses, top_n=10):
    ##### Collaborative Based Filtering using Item Similarity
    item_based_recommendations = item_based_recommend_events(user, top_n)
    recommended_events = pd.DataFrame(item_based_recommendations, columns=['Event ID', 'score'])

    # Merge with events_df to get event details
    recommended_events = recommended_events.merge(events_df, on='Event ID')
    
    ##### Content Based Filtering
    responses = responses[responses['Name'] == user]
    responses['All Interests'] = responses['All Interests'].str.split(', ')
    events_df['Category'] = events_df[['Primary Category', 'Secondary Category']].apply(lambda x: ', '.join(filter(lambda y: pd.notna(y), x)), axis=1)
    events_df['Category'] = events_df['Category'].str.split(', ')
    events_df['Interest_Match'] = False

    for index, row in responses.iterrows():
        for tag in row['All Interests']:
            events_df.loc[events_df['Category'].apply(lambda x: tag in x), 'Interest_Match'] = True

    events_df = events_df[events_df['Interest_Match'] == True]

    interacted_events = interactions_df[interactions_df['Name'] == user]
    filtered_interactions_df_1 = interacted_events[~interacted_events['Interaction Score'].isin([-1, 0])]
    merged_df = pd.merge(interacted_events, events_df[['Event ID', 'Primary Category', 'Sub Category']], on='Event ID', how='left', suffixes=('_int', '_evt'))
    merged_df_1 = pd.merge(filtered_interactions_df_1, events_df[['Event ID', 'Primary Category', 'Sub Category']], on='Event ID', how='left', suffixes=('_int', '_evt'))
    filtered_interactions_df_neg_zero = interacted_events[interacted_events['Interaction Score'].isin([-1, 0])]

    categories = merged_df.sort_values('Interaction Score', ascending=False)['Primary Category_evt'].unique()
    recommended_categories = set()

    for category in categories:
        category_events = events_df[events_df['Primary Category'] == category]
        user_subcategories = merged_df_1[merged_df_1['Primary Category_evt'] == category]['Sub Category_evt'].unique()
        for subcategory in user_subcategories:
            subcategory_events = category_events[category_events['Sub Category'] == subcategory]
            new_events = subcategory_events[~subcategory_events['Event ID'].isin(merged_df_1['Event ID']) &
                                            ~subcategory_events['Event ID'].isin(recommended_events['Event ID']) &
                                            ~subcategory_events['Event ID'].isin(filtered_interactions_df_neg_zero['Event ID'])]
            if not new_events.empty:
                recommended_categories.add(category)
                recommended_events = pd.concat([recommended_events, new_events], ignore_index=True)

    # Add unrecommended events
    unrecommended_categories = set(categories) - recommended_categories
    for ur_category in unrecommended_categories:
        category_events_1 = events_df[events_df['Primary Category'] == ur_category]
        user_subcategories_1 = merged_df[merged_df['Primary Category_evt'] == ur_category]['Sub Category_evt'].unique()
        for sc in user_subcategories_1:
            subcategory_events_1 = category_events_1[category_events_1['Sub Category'] != sc]
            ur_category_events = subcategory_events_1[
                (~subcategory_events_1['Event ID'].isin(merged_df['Event ID'])) &
                (~subcategory_events_1['Event ID'].isin(recommended_events['Event ID']))]
            recommended_events = pd.concat([recommended_events, ur_category_events], ignore_index=True)

    # Add additional events based on sorting
    sorted_events = merged_df.sort_values('Interaction Score', ascending=False)
    for index, event in sorted_events.iterrows():
        additional_events = events_df[
            (events_df['Primary Category'] == event['Primary Category_evt']) &
            (~events_df['Event ID'].isin(merged_df['Event ID'])) &
            (~events_df['Event ID'].isin(recommended_events['Event ID']))]
        recommended_events = pd.concat([recommended_events, additional_events], ignore_index=True)

    recommended_events.drop('score', axis=1, inplace=True)
    return recommended_events

user = 'Pratik Bhande'
recommended_events = recommend_events(user, events_df, responses)
recommended_events.head(10)


C:\Users\pratik\AppData\Local\Temp\ipykernel_6772\4110593307.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  responses['All Interests'] = responses['All Interests'].str.split(', ')


,Event ID,Event Names,Description,Primary Category,Secondary Category,Sub Category,Location,Prices,Date,Unnamed: 9,Time,Links,Category,Interest_Match
0,EVT0030,Matheran Forest Camping,NaN,Travel,NaN,Camping,Sangamvadi,1499,2024-02-06 00:00:00,Tuesday,19:00:00,NaN,NaN,NaN
1,EVT0085,Dandeli Jungle Tour,उनाड भ्रमंती is happy to invite you for *Dande...,Adventure,Travel,Hiking and Trekking,Sangamvadi,10000,2024-03-29 00:00:00,Friday,00:00:00,https://allevents.in/pune/dandeli-jungle-tour-...,NaN,NaN
2,EVT0122,NutriWell Business Conclave,Nutriwell Business Conclave organised by ETHea...,Fitness,NaN,Nutrition,Bavdhan,499,2024-07-05 00:00:00,NaN,05:00:00,NaN,NaN,NaN
3,EVT0147,Flea Market - local shopping,Explore innovative startups and entrepreneurs ...,Music and Entertainment,Food and Drinks,Shopping,Karve Nagar,50,2024-04-13 00:00:00,NaN,17:00:00,NaN,NaN,NaN
4,EVT0022,Fun and Fair,"Don’t miss new attractions, the thrills of the...",Music and Entertainment,Food and Drinks,Exhibition,Erandwane,0,23/3/24 to 31/3/24,23/3/24 to 31/3/24,15:00:00,NaN,NaN,NaN
5,EVT0131,Ambenali Climbing Challenge,Join us for one of the simplest and safest cli...,Adventure,NaN,Adventure Sports,Shivajinagar,1800,2024-04-14 00:00:00,NaN,06:00:00,https://allevents.in/mahabaleswar/ambenali-cli...,NaN,NaN
6,EVT0081,Kolad River Side Camping and Water Sports,*Dear people* Adventure Camp Zone ( Kolad )Get...,Adventure,Travel,Adventure Sports,Deccan,1500,2024-03-31 00:00:00,Sunday,15:00:00,https://allevents.in/roha/kolad-river-side-cam...,NaN,NaN
7,EVT0188,Train for Mental Fitness,Discover the root of stress and other negative...,Fitness,NaN,Meditation and Mindfulness,Viman Nagar,0,2024-04-07 00:00:00,NaN,11:00:00,NaN,NaN,NaN
8,EVT0140,Regional Scrum Gathering Conference,We are thrilled to announce the Regional Scrum...,Business and Networking,NaN,Conference,Kharadi,NaN,2024-06-04 00:00:00,NaN,09:00:00,NaN,NaN,NaN
9,EVT0041,Cricket Tournament,NaN,Sports,NaN,Cricket,Wagholi,600,2024-11-12 00:00:00,Tuesday,10:00:00,NaN,NaN,NaN
